In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders.binary import BinaryEncoder
import seaborn as sns

In [2]:
df = pd.read_csv(r"data/sessions.csv", index_col=0);

C:\Users\mirzo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.shape

(216801, 11)

In [5]:
df.head()

,utm_source,utm_medium,utm_campaign,utm_adcontent,device_category,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,is_target_action
0,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Huawei,360x720,Chrome,Russia,Zlatoust,0
1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,mobile,Samsung,385x854,Samsung Internet,Russia,Moscow,0
2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Huawei,360x720,Chrome,Russia,Krasnoyarsk,0
5,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Apple,375x812,Safari,Russia,Saint Petersburg,0
6,TxKUcPpthBDPieTGmVhx,cpc,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,tablet,Lenovo,602x1029,YaBrowser,Russia,Saint Petersburg,0


In [6]:
df.columns

Index(['utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'device_category', 'device_brand', 'device_screen_resolution',
       'device_browser', 'geo_country', 'geo_city', 'is_target_action'],
      dtype='object')

In [5]:
def create_new_feat(df: pd.DataFrame, inplace=False):
    df_new = df if inplace else df.copy()
    df_new['screen_area'] = df_new.device_screen_resolution.apply(lambda screen: np.prod([int(size) for size in screen.split('x')]))
    df_new = df_new.drop(['device_screen_resolution'], axis=1)
    return df_new

In [6]:
def scale_numeric_feat(df: pd.DataFrame, inplace=False):
    df_new = df if inplace else df.copy()
    scaler = StandardScaler()
    df_new['screen_area_scl'] = scaler.fit_transform(df_new[['screen_area']])
    df_new = df_new.drop(['screen_area'], axis=1)
    return df_new

In [7]:
def category_coder(df: pd.DataFrame, inplace=False):
    df_new = df if inplace else df.copy()
    encoder = BinaryEncoder()
    obj_cols = list(df_new.dtypes[df_new.dtypes == 'object'].index)
    df_obj_cols = df_new[obj_cols]
    df_new.drop(obj_cols, axis=1, inplace=True)
    encoder.fit(df_obj_cols)
    df_new[encoder.get_feature_names_out()] = encoder.transform(df_obj_cols)
    return df_new

In [37]:
def preprocess(df: pd.DataFrame, inplace=False):
    df_new = df if inplace else df.copy()     
    
    df_new = create_new_feat(df_new, inplace=inplace)
    print("create_new_feat")
    
    df_new = scale_numeric_feat(df_new, inplace=inplace)
    print("scale_numeric_feat")
    
    df_new = category_coder(df_new, inplace=inplace)
    print("category_coder")
    return df_new

Конверсия 2.9%

In [9]:
df_old = df.copy()

In [36]:
df = df_old.copy()

In [39]:
df = preprocess(df, inplace=True)

delete_data
create_new_feat
scale_numeric_feat
category_coder


In [45]:
df.to_csv(r"data/session_prepared_binCoder.csv")